<a href="https://colab.research.google.com/github/SemihDemir13/chatbot-project-data/blob/main/V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1
print("Gerekli kütüphaneler kuruluyor...")
!pip install -q -U transformers huggingface_hub accelerate sentence-transformers torch langchain langchain-community langchain-huggingface chromadb pandas
print("Kurulum tamamlandı.")

In [ ]:
#2
from huggingface_hub import login

# Token'ını girdikten sonra 'Login successful' yazısını görmelisin.
login()

In [ ]:
#3
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

print("Model yükleniyor... (Biraz sabır)")

model_id = "google/gemma-2b-it"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    # Colab GPU'su kullanılsın diye device_map="auto" diyoruz
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype="auto"
    )

    # Text generation pipeline'ı oluşturuyoruz
    llm_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=250 # Cevabın uzunluğu
    )
    print("✅ Model başarıyla yüklendi!")

except Exception as e:
    print(f"❌ HATA: Model yüklenemedi. Lütfen HÜCRE 2'de token girdiğinizden ve modelin lisansını kabul ettiğinizden emin olun.\nHata: {e}")


In [ ]:
#4
import pandas as pd
import sys

print("CSV dosyası okunuyor ve veri formatlanıyor...")

try:
    df = pd.read_csv('netflix_titles.csv')
    df = df.fillna('Bilinmiyor')

    dokumanlar = []

    # Tüm veriyi alıyoruz
    print(f"Toplam {len(df)} adet kayıt işleniyor...")

    for index, row in df.iterrows():
        # Formatı sadeleştirdik. Başlık en üstte.
        metin = (
            f"Başlık: {row['title']}\n"
            f"Yönetmen: {row['director']}\n"
            f"Yıl: {row['release_year']}\n"
            f"Tür: {row['listed_in']}\n"
            f"Özet: {row['description']}\n"
            f"Detay: Bu film {row['release_year']} yılında yayınlanmıştır. {row['type']} kategorisindedir."
        )
        dokumanlar.append(metin)

    print(f"✅ {len(dokumanlar)} adet içerik başarıyla hazırlandı.")

except FileNotFoundError:
    print("❌ HATA: Dosya yok.")

In [ ]:
#5
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

print("Vektör veritabanı (Çok Dilli) oluşturuluyor...")

# DEĞİŞİKLİK BURADA:
# Artık Türkçe-İngilizce eşleştirmesi yapabilen 'multilingual' modeli kullanıyoruz.
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# ChromaDB veritabanını kur
# Eğer hata alırsan önceki veritabanını temizlemek gerekebilir ama genellikle üzerine yazar.
vektor_veritabani = Chroma.from_texts(dokumanlar, embedding=embedding_model)

print("✅ Çok dilli veritabanı hazır! Artık Türkçe sorguları İngilizce verilerle eşleştirebilirim.")

In [ ]:
import re
import random

def temizle_yorumlar(metin):
    """Modelin sonuna eklediği İngilizce yorumları ve bozuk kısımları siler."""
    yasakli_cumleler = [
        "I hope this", "Please note", "This phrase", "The phrase means",
        "This is a", "Translated", "Here is", "What does", "English:", "Turkish:"
    ]
    satirlar = metin.split('\n')
    temiz_satirlar = []
    for satir in satirlar:
        if any(yasak in satir for yasak in yasakli_cumleler):
            break
        if satir.strip():
            temiz_satirlar.append(satir)
    return " ".join(temiz_satirlar).strip()

def rag_yanit(soru, top_k=3):
    # --- HAZIRLIK ---
    tur_ceviri = {
        "korku": "Horror", "gerilim": "Thriller", "aksiyon": "Action",
        "komedi": "Comed", "drama": "Dram", "romantik": "Romantic", "aşk": "Romantic",
        "bilim kurgu": "Sci-Fi", "belgesel": "Documentary", "çocuk": "Children",
        "aile": "Family", "anime": "Anime", "macera": "Adventure", "suç": "Crime"
    }

    etkisiz_kelimeler = ["filmi", "film", "öner", "bana", "hakkında", "olan", "yapımı", "yönetmen", "oyuncu", "kimdir", "listesi", "var", "mı", "mu", "bir", "söyle", "istiyorum", "tavsiye", "et", "izle"]

    temiz_soru = soru.lower()
    for kelime in etkisiz_kelimeler:
        temiz_soru = temiz_soru.replace(f" {kelime} ", " ").replace(f" {kelime}", "").replace(f"{kelime} ", "")

    anahtar_kelime = temiz_soru.strip()
    anahtar_kelimeler_listesi = anahtar_kelime.split()

    secilen_film_bilgisi = {}
    kaynak_yontem = ""
    uygun_filmler = pd.DataFrame()

    # --- 1. ADIM: ANALİZ ---
    yil_tespiti = re.search(r'(\d{4})', soru)
    min_yil = int(df['release_year'].min())
    max_yil = int(df['release_year'].max())

    # Tür Tespiti
    aranan_tur_eng = None
    for tr_tur, eng_tur in tur_ceviri.items():
        if tr_tur in soru.lower():
            aranan_tur_eng = eng_tur
            break

    # --- 2. ADIM: ARAMA SENARYOLARI ---

    # SENARYO A: YIL VARSA
    if yil_tespiti:
        aranan_yil = int(yil_tespiti.group(1))
        if aranan_yil < min_yil or aranan_yil > max_yil:
            return f"Üzgünüm, veri setimde sadece {min_yil} ile {max_yil} yılları arasındaki filmler bulunmaktadır. ({aranan_yil} mevcut değil)"

        kaynak_yontem = f"YIL FİLTRESİ ({aranan_yil})"
        uygun_filmler = df[df['release_year'] == aranan_yil]

        # Yılın içinde Tür de varsa süz
        if aranan_tur_eng and not uygun_filmler.empty:
            tur_filtreli = uygun_filmler[uygun_filmler['listed_in'].str.contains(aranan_tur_eng, case=False, na=False)]
            if not tur_filtreli.empty:
                uygun_filmler = tur_filtreli
                kaynak_yontem += f" + TÜR ({aranan_tur_eng})"

    # SENARYO B: SADECE TÜR VARSA (YENİ EKLENDİ!)
    # Eğer yıl yoksa ama kullanıcı "Korku filmi" dediyse buraya girecek.
    elif aranan_tur_eng:
        kaynak_yontem = f"TÜR FİLTRESİ ({aranan_tur_eng})"
        uygun_filmler = df[df['listed_in'].str.contains(aranan_tur_eng, case=False, na=False)]

    # SENARYO C: İSİM VARSA (Yıl ve Tür yoksa isimdir)
    elif len(anahtar_kelime) > 2:
        temp_df_cast = df.copy()
        temp_df_dir = df.copy()
        for kelime in anahtar_kelimeler_listesi:
            temp_df_cast = temp_df_cast[temp_df_cast['cast'].str.contains(kelime, case=False, na=False)]
            temp_df_dir = temp_df_dir[temp_df_dir['director'].str.contains(kelime, case=False, na=False)]

        uygun_filmler = pd.concat([temp_df_cast, temp_df_dir]).drop_duplicates()
        if not uygun_filmler.empty:
            kaynak_yontem = f"İSİM FİLTRESİ ('{anahtar_kelime}')"

    # --- 3. ADIM: VERİYİ SEÇME ---
    if not uygun_filmler.empty:
        secilen = uygun_filmler.sample(n=1).iloc[0]
        secilen_film_bilgisi = {
            "title": secilen['title'],
            "year": secilen['release_year'],
            "desc": secilen['description'],
            "genre": secilen['listed_in']
        }
        print(f"\n[SİSTEM: {kaynak_yontem} bulundu: {secilen['title']}]")

    # SENARYO D: VEKTÖR ARAMASI (Son Çare)
    else:
        if yil_tespiti: return f"Aradığınız kriterlere uygun film bulunamadı."

        kaynak_yontem = "VEKTÖR ARAMASI"
        docs = vektor_veritabani.similarity_search(soru, k=1)
        if docs:
            doc = docs[0]
            lines = doc.page_content.split('\n')
            title, desc, year, genre = "Bilinmiyor", "Açıklama yok.", "???", "???"
            for line in lines:
                if "Başlık:" in line: title = line.replace("Başlık: ", "").strip()
                if "Özet:" in line or "Description:" in line: desc = line.split(":", 1)[1].strip()
                if "Yıl:" in line: year = line.replace("Yıl: ", "").strip()
                if "Tür:" in line: genre = line.replace("Tür: ", "").strip()

            secilen_film_bilgisi = {"title": title, "year": year, "desc": desc, "genre": genre}
            print(f"\n[SİSTEM: Vektörden bulundu: {title}]")

    print("-" * 40)

    # --- 4. ADIM: ÇEVİRİ ---
    if not secilen_film_bilgisi: return "Üzgünüm, kriterlerinize uygun bir film bulamadım."

    ingilizce_ozet = secilen_film_bilgisi['desc']
    prompt = f"Task: Translate the text below into fluent Turkish. Do not explain. Text: {ingilizce_ozet}\nTranslation:"

    try:
        sonuc = llm_pipeline(prompt, return_full_text=False, max_new_tokens=100)[0]["generated_text"]
        turkce_ozet = temizle_yorumlar(sonuc.strip())
        if len(turkce_ozet) < 5: turkce_ozet = ingilizce_ozet
    except:
        turkce_ozet = ingilizce_ozet

    # --- 5. ADIM: CEVAP ---
    final_cevap = (f"Önerim: {secilen_film_bilgisi['title']} ({secilen_film_bilgisi['year']})\n"
                   f"Tür: {secilen_film_bilgisi['genre']}\n"
                   f"Konusu: {turkce_ozet}")

    return final_cevap

In [ ]:
print("\n================================================")
print("🎬 NETFLIX GURUSU (Çıkmak için 'çıkış' yazın)")
print("================================================\n")

while True:
    soru = input("Soru sor: ")
    if soru.lower() in ['çıkış', 'exit', 'quit']:
        print("İyi seyirler 👋")
        break

    print("🔎 Araştırıyorum...")
    cevap = rag_yanit(soru)
    print(f"\n🤖 AI: {cevap}\n")
    print("-" * 40)
